# A New Condition for Agglomeration in Bayesian Confirmation

This notebook corresponds to the following paper:

- https://www.cambridge.org/core/journals/philosophy-of-science/article/new-condition-for-agglomeration-in-bayesian-confirmation/BF3906892462EEE562594F1F2713C119

## Importing Packages and Setup

In [1]:
import numpy as np
import pandas as pd
import polars as pl
from scipy import stats
from itertools import chain, combinations
from IPython.display import Math, display
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Defining Functions for Powerset and Probabilities

In [2]:
# Powerset function (without the empty set)
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

# Function that calculates all marginal and joint probabilities for a set of probability distributions, given as matrix
def probabilities(matrix):

    # Size of matrix, number of random variables
    length, width = matrix.shape
    num_variables = np.log2(width).astype(int)
    
    # Create matrix with all Boolean combinations
    binmat = np.array([np.fromstring(np.binary_repr(i,num_variables), dtype=np.uint8)==49 for i in range(0,width)])

    # Pick all probabilities from matrix
    result = [pd.Series(np.sum(np.all(binmat[:,i],axis=1)*matrix,axis = 1), name = f"P{i}") for i in powerset(range(num_variables))]

    return pl.DataFrame(result)

## Generating probability functions

In [3]:
# Number of variables and distributions
number_variables = 3
number_distributions = 10000000

# Sampling from Dirichlet distribution, calculating relevant probabilities
matrix = pl.DataFrame(np.random.dirichlet(np.ones(2**number_variables), number_distributions))
df = probabilities(matrix)

## Results

In [4]:
# Conjuncts are confirmed (condition 1)
conjuncts_confirmation = (df["P(0, 1)"] > df["P(0,)"]*df["P(1,)"]) & (df["P(0, 2)"] > df["P(0,)"]*df["P(2,)"])
conjuncts_confirmation_mean = conjuncts_confirmation.mean()

# Conjunction is confirmed (condition 2)
conjunction_confirmation = (df["P(0, 1, 2)"] > df["P(0,)"]*df["P(1, 2)"])
conjunction_confirmation_mean = conjunction_confirmation.mean()

# NOR-confirmation is satisfied (condition 3)
nor_confirmation = df["P(0, 1)"] + df["P(0, 2)"] - df["P(0, 1, 2)"] < df["P(0,)"] * (df["P(1,)"] + df["P(2,)"] - df["P(1, 2)"] )
nor_confirmation_mean = nor_confirmation.mean()

# NOR-Effect
nor_effect = (conjuncts_confirmation & nor_confirmation)
nor_effect_conjunctive_prevalence = nor_effect.mean()
nor_effect_conditional_prevalence = nor_effect_conjunctive_prevalence/conjuncts_confirmation_mean

# Simpson's Paradox
simpson_one = (df["P(0, 1, 2)"] > df["P(1, 2)"] * df["P(0, 2)"]/df["P(2,)"])
simpson_two = (df["P(0, 1)"] - df["P(0, 1, 2)"] > (df["P(1,)"] - df["P(1, 2)"]) * (df["P(0,)"] - df["P(0, 2)"])/(1-df["P(2,)"])  )
simpson_three = (df["P(0, 1)"] <= df["P(0,)"]*df["P(1,)"])
simpson_paradox_conjunctive = (simpson_one & simpson_two & simpson_three).mean()

simpson_antecedent_mean = (simpson_one & simpson_two).mean()
simpson_paradox_conditional = simpson_paradox_conjunctive / simpson_antecedent_mean

# Collecting results
results = pd.DataFrame([
    [nor_effect_conjunctive_prevalence, nor_effect_conditional_prevalence],
    [simpson_paradox_conjunctive, simpson_paradox_conditional],
]).rename(columns={0: "Conjunctive Prevalence", 1: "Conditional Prevalence"})

effects = pd.DataFrame([
['NOR-confirmation Effect'],
["Simpson's Paradox"],
]).rename(columns={0: "Effect"})

pd.concat([effects,results],axis=1).style.hide()

Effect,Conjunctive Prevalence,Conditional Prevalence
NOR-confirmation Effect,0.024999,0.100013
Simpson's Paradox,0.008344,0.033371
